![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)


# CAPSTONE PROJECT: Predicción de precios de las viviendas en Madrid
Limpieza de datos: Pisos.com

Meta de creación de columnas:
- `fecha_descarga`
- `web`
- `url`
- `precio`
- `barrio`
- `distrito`
- `codigo_postal`
- `metros_utiles`
- `metros_construidos`
- `num_hab`
- `num_banos`
- `planta`
- `tipologia`
- `obra_nueva`
- `cocina`
- `garage`
- `trastero`
- `orientacion`
- `estado`
- `antiguedad`
- `consumo`
- `fecha_publicacion`
- `subasta`
- `visitas`
- `amueblado`
- `calefaccion`
- `terraza`
- `balcon`
- `piscina`
- `jardin`
- `aire_acondionado`
- `detalle`

Dos datasets recibidos
- Detalles_datasales19102024.xlsx
- Detalles2_datasales19102024.xlsx

In [468]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import unidecode

pd.options.plotting.backend = 'plotly' # Para ejecutar las gráficas con Plotly y no Matpolotlib.

# Ajustar pandas para mostrar contenido completo
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_colwidth')

# Explorar los dataframes de "detalles"

### TRANSFORMAR COLUMNAS Y MERGE

In [469]:
df_detalles1 = pd.read_excel('./03-pisos-com//Detalles_datasales19102024_referencia.xlsx')
df_detalles1.tail(1)

,Campo1,Campo2,Título,Campo3,Campo4,Campo5,Campo6,Campo7,Campo8,Campo9_texto,Campo9_enlaces,Campo10,Campo11,Campo12,Campo13,Campo14,Campo15,Campo16,Campo17,Campo18,Campo19
2865,3.300.000 €,3 habs.\n\t\t\t6 baños\n\t\t\t301 m²\n\t\t\t2ª planta\n\t\t\t10.963 €/m²,Piso en venta en Prosperidad,El Viso (Distrito Chamartín. Madrid Capital),"¡Exclusivo Piso de Lujo en El Viso, Madrid! Presentamos una oportunidad única para los amantes del lujo: un exclusivo piso en uno de los barrios más prestigiosos de Madrid, El Viso. Este moderno edificio, diseñado por el reconocido arquitecto Joaquín Torres, cuenta con solo 8 viviendas, garantizando privacidad y exclusividad. El piso ocupa toda la planta, por lo que no tiene vecinos laterales. El ascensor privado ofrece acceso directo a un recibidor con aseo de cortesía, permitiendo entrar de manera independiente a la cocina, al salón o al área de dormitorios. Distribución: La elegante y luminosa cocina en isla tiene acceso al comedor y a la terraza, se encuentra completamente equipada con electrodomésticos de alta gama, tiene un con baño, lavadero y zona de servicio. El imponente Salón-Comedor es un espacio elegante pero a la vez cálido y acogedor, con mucha luz y con acceso a una magnífica terraza con impresionantes vistas. A la zona de dormitorios se accede desde el salón o desde el recibidor a través de un pasillo que le brinda intimidad con su acceso independiente. El primero de estos tiene dos armarios grandes y un baño en suite con plato de ducha. El segundo dormitorio tiene un vestidor y baño en suite con bañera de hidromasajes. El majestuoso dormitorio principal tiene un baño con plato de ducha, un vestidor con baño con bañera de hidromasajes y otro vestidor. Este singular piso es totalmente exterior, con abundante luz natural incluso en los días más nublados, sus calidades constructivas son materiales de primera, aire acondicionado central y calefacción por aerotermia. Ventanas Climalit de doble cristal con excelente aislamiento térmico y acústico en todas sus estancias. Tiene dos amplias plazas de garaje. El Viso es una zona consolidada con viviendas de alto standing, embajadas y consulados, lo que garantiza la mayor seguridad por contar con vigilancia policial permanente. Dispone de reconocidos restaurantes, el Hospital Universitario San Francisco de Asís, el EAE Business School, supermercados, y está muy bien comunicada con calles emblemáticas como Velázquez, Serrano, López de Hoyos, Paseo de la Castellana y Príncipe de Vergara. ¡No pierda la oportunidad de vivir en este enclave de lujo y exclusividad! Contáctenos para más información y para concertar una visita.",Mostrar más,Características\n\t\t\n Datos básicos\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie construida: \n\t\t\t\t\t\t\t\t\t\t301 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t3\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t6\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tPlanta: \n\t\t\t\t\t\t\t\t\t\t2ª\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tExterior\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tGastos de comunidad: \n\t\t\t\t\t\t\t\t\t\tMás de 100 €\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\tXA856-1025\n\t\t\t\t\t\t\t\t\n \n \n Equipamiento e instalaciones\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAgua\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAire acondicionado: \n\t\t\t\t\t\t\t\t\t\tFrío y calor\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tCalefacción: \n\t\t\t\t\t\t\t\t\t\tEléctrica\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tCocina equipada\n\t\t\t\t\t\t\t\t\n \n

In [470]:
df_detalles2 = pd.read_excel('./03-pisos-com//Detalles2_datasales19102024_referencia.xlsx')

In [471]:
#df_detalles2 = pd.read_excel('/Users/AleVC/Library/CloudStorage/GoogleDrive-ale.vaquero.c@gmail.com/My Drive/Nuclio/Máster en Data Science/TFM/PisosCom/Detalles2_datasales19102024.xlsx')
df_detalles2.head(1)

,Campo1,Campo2,Campo3,Campo4,Campo5,Campo6,Título,Campo7,Campo8,Campo9,Campo10,Tiempo,Campo11,Tiempo1,Campo12,Campo13,Campo14,Campo15
0,1.900.000 €,5 habs.,6 baños,780 m²,2.435 €/m²,NaN,"Chalet en venta en Calle de Arroyofresno, 10",Ciudad Universitaria (Distrito Moncloa-Aravaca. Madrid Capital),"¿La casa de tus sueños es está quieres visitarla?La mejor zona de Madrid a 5 minutos de Moncloa acceso a M30, M40, A6, Aeropuerto a 12 minutos. La ubicación dentro de Madrid Puerta de Hierro, sentirás paz tranquilidad, pajaritos, zonas verdes, aire limpio contamos con un Sánchez Romero a 2 minutos, Mercadona a 4 minutos, Farmacia a 2 minutos, Bancos a 2 minutos, Colegios a 3 minutos, Transporte público a 1 minuto. Mi casa ahora, ¿mañana puede ser tuya? Construcción 2018, 413 m2, domótica en toda la casa placas solares, suelo radiante,y más. Con una piscina preciosa, porche, jardín, 4 habitaciones, 6 baños, una suite principal con un gran vestidor baño terraza y todas las habitaciones con baño privado y terrazas. Muy luminoso. Garaje privado para 3 - 4 coches, ascensor privado.¿Cuándo quieras venir a visitarla? disponibilidad tardes, directamente propietarios. Mas fotos privado. 1.945.000",Superficie construida: \n\t\t\t\t\t\t\t\t\t\t780 m²,Superficie construida: \n\t\t\t\t\t\t\t\t\t\t780 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie útil: \n\t\t\t\t\t\t\t\t\t\t452 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie solar: \n\t\t\t\t\t\t\t\t\t\t780 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t5\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t6\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAntigüedad: \n\t\t\t\t\t\t\t\t\t\tMenos de 5 años\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tConservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tGastos de comunidad\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\t21485502,Superficie solar: \n\t\t\t\t\t\t\t\t\t\t780 m²,Habitaciones: \n\t\t\t\t\t\t\t\t\t\t5,Superficie útil: \n\t\t\t\t\t\t\t\t\t\t452 m²,Clasificación: Disponible,Certificado energético,Anuncio actualizado el 23/10/2024,Ref.: -1


In [472]:
df_detalles1.shape
# (2868, 21)

(2866, 21)

In [473]:
df_detalles2.shape
# (2807, 18)

(2805, 18)

In [474]:
df_detalles1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2866 entries, 0 to 2865
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Campo1          2866 non-null   object 
 1   Campo2          2860 non-null   object 
 2   Título          2619 non-null   object 
 3   Campo3          2619 non-null   object 
 4   Campo4          2866 non-null   object 
 5   Campo5          2789 non-null   object 
 6   Campo6          2866 non-null   object 
 7   Campo7          2715 non-null   object 
 8   Campo8          2866 non-null   object 
 9   Campo9_texto    2866 non-null   object 
 10  Campo9_enlaces  2866 non-null   object 
 11  Campo10         2866 non-null   object 
 12  Campo11         2866 non-null   object 
 13  Campo12         0 non-null      float64
 14  Campo13         0 non-null      float64
 15  Campo14         0 non-null      float64
 16  Campo15         0 non-null      float64
 17  Campo16         0 non-null      f

In [475]:
df_detalles2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2805 entries, 0 to 2804
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Campo1   2805 non-null   object
 1   Campo2   2799 non-null   object
 2   Campo3   2799 non-null   object
 3   Campo4   2790 non-null   object
 4   Campo5   2710 non-null   object
 5   Campo6   1767 non-null   object
 6   Título   2564 non-null   object
 7   Campo7   2564 non-null   object
 8   Campo8   2805 non-null   object
 9   Campo9   2805 non-null   object
 10  Campo10  2805 non-null   object
 11  Tiempo   2797 non-null   object
 12  Campo11  2762 non-null   object
 13  Tiempo1  2804 non-null   object
 14  Campo12  2805 non-null   object
 15  Campo13  2805 non-null   object
 16  Campo14  2805 non-null   object
 17  Campo15  2805 non-null   object
dtypes: object(18)
memory usage: 394.6+ KB


In [476]:
#df_detalles1 = df_detalles1.drop(['Campo5', 'Campo7', 'Campo9_texto', 'Campo12', 'Campo13', 'Campo14', 'Campo15', 'Campo16', 'Campo17', 'Campo18', 'Campo19'], axis=1)
#df_detalles1.shape
# 2868, 10

In [477]:
#df_detalles2 = df_detalles2.drop(['Tiempo', 'Campo9', 'Campo11', 'Tiempo1', 'Campo13'], axis=1)
#df_detalles2.shape
# (2807, 13)

In [478]:
df_detalles1.columns

Index(['Campo1', 'Campo2', 'Título', 'Campo3', 'Campo4', 'Campo5', 'Campo6',
       'Campo7', 'Campo8', 'Campo9_texto', 'Campo9_enlaces', 'Campo10',
       'Campo11', 'Campo12', 'Campo13', 'Campo14', 'Campo15', 'Campo16',
       'Campo17', 'Campo18', 'Campo19'],
      dtype='object')

In [479]:
#df_detalles2.columns

In [480]:
df_detalles1 = df_detalles1.rename(columns={'Campo1': 'precio', 'Campo2': 'espacios', 'Título': 'title', 'Campo3': 'distrito', 'Campo4': 'detalle', 'Campo6': 'caracteristicas', 'Campo8': 'consumo', 'Campo9_enlaces': 'url', 'Campo10': 'actualizacion', 'Campo11': 'referencia'})
df_detalles1.tail(1)

,precio,espacios,title,distrito,detalle,Campo5,caracteristicas,Campo7,consumo,Campo9_texto,url,actualizacion,referencia,Campo12,Campo13,Campo14,Campo15,Campo16,Campo17,Campo18,Campo19
2865,3.300.000 €,3 habs.\n\t\t\t6 baños\n\t\t\t301 m²\n\t\t\t2ª planta\n\t\t\t10.963 €/m²,Piso en venta en Prosperidad,El Viso (Distrito Chamartín. Madrid Capital),"¡Exclusivo Piso de Lujo en El Viso, Madrid! Presentamos una oportunidad única para los amantes del lujo: un exclusivo piso en uno de los barrios más prestigiosos de Madrid, El Viso. Este moderno edificio, diseñado por el reconocido arquitecto Joaquín Torres, cuenta con solo 8 viviendas, garantizando privacidad y exclusividad. El piso ocupa toda la planta, por lo que no tiene vecinos laterales. El ascensor privado ofrece acceso directo a un recibidor con aseo de cortesía, permitiendo entrar de manera independiente a la cocina, al salón o al área de dormitorios. Distribución: La elegante y luminosa cocina en isla tiene acceso al comedor y a la terraza, se encuentra completamente equipada con electrodomésticos de alta gama, tiene un con baño, lavadero y zona de servicio. El imponente Salón-Comedor es un espacio elegante pero a la vez cálido y acogedor, con mucha luz y con acceso a una magnífica terraza con impresionantes vistas. A la zona de dormitorios se accede desde el salón o desde el recibidor a través de un pasillo que le brinda intimidad con su acceso independiente. El primero de estos tiene dos armarios grandes y un baño en suite con plato de ducha. El segundo dormitorio tiene un vestidor y baño en suite con bañera de hidromasajes. El majestuoso dormitorio principal tiene un baño con plato de ducha, un vestidor con baño con bañera de hidromasajes y otro vestidor. Este singular piso es totalmente exterior, con abundante luz natural incluso en los días más nublados, sus calidades constructivas son materiales de primera, aire acondicionado central y calefacción por aerotermia. Ventanas Climalit de doble cristal con excelente aislamiento térmico y acústico en todas sus estancias. Tiene dos amplias plazas de garaje. El Viso es una zona consolidada con viviendas de alto standing, embajadas y consulados, lo que garantiza la mayor seguridad por contar con vigilancia policial permanente. Dispone de reconocidos restaurantes, el Hospital Universitario San Francisco de Asís, el EAE Business School, supermercados, y está muy bien comunicada con calles emblemáticas como Velázquez, Serrano, López de Hoyos, Paseo de la Castellana y Príncipe de Vergara. ¡No pierda la oportunidad de vivir en este enclave de lujo y exclusividad! Contáctenos para más información y para concertar una visita.",Mostrar más,Características\n\t\t\n Datos básicos\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie construida: \n\t\t\t\t\t\t\t\t\t\t301 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t3\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t6\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tPlanta: \n\t\t\t\t\t\t\t\t\t\t2ª\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tExterior\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tGastos de comunidad: \n\t\t\t\t\t\t\t\t\t\tMás de 100 €\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\tXA856-1025\n\t\t\t\t\t\t\t\t\n \n \n Equipamiento e instalaciones\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAgua\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAire acondicionado: \n\t\t\t\t\t\t\t\t\t\tFrío y calor\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tCalefacción: \n\t\t\t\t\t\t\t\t\t\tEléctrica\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tCocina equipada\n\t\t\t\t\

In [481]:
df_detalles1.query("referencia=='Ref.: -0'")

,precio,espacios,title,distrito,detalle,Campo5,caracteristicas,Campo7,consumo,Campo9_texto,url,actualizacion,referencia,Campo12,Campo13,Campo14,Campo15,Campo16,Campo17,Campo18,Campo19
1,2.950.000 €,Obra nueva\n\n\t\t\t7 habs.\n\t\t\t8 baños\n\t\t\t561 m²\n\t\t\t5.258 €/m²,"Chalet en venta en Calle del Pico Milano, s/n",Mirasierra-Arroyo del Fresno (Distrito Fuencarral-El Pardo. Madrid Capital),"Parcela 2, Tipo B. Chalet de obra nueva de 561 m2 de superficie construida y situado en la zona de Madrid Capital. Está estructurado en 7 habitaciones, 8 baños completos y terraza. El Chalet tiene piscina. Aporta como extras garaje y trastero.",Mostrar más,Características\n\t\t\n Datos básicos\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie construida: \n\t\t\t\t\t\t\t\t\t\t561 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t7\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t8\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tConservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\t-0\n\t\t\t\t\t\t\t\t\n \n \n Equipamiento e instalaciones\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tGaraje\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tTrastero\n\t\t\t\t\t\t\t\t\n \n \n Exteriores\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tTerraza\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tJardín: \n\t\t\t\t\t\t\t\t\t\tPrivado\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tPiscina\n\t\t\t\t\t\t\t\t\n \n \n \n\t\tVer todas las características\n\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\tCaracterísticas,Ver todas las características,"Certificado energético\n\n Clasificación: En trámite\n\n\n\n Ver detalle \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n Certificado energético\n \n \n La certificación energética indica la eficiencia del inmueble en cuanto a consumo de energía, agua, etc. La categoría A es la más eficiente y la G la menos.\n \n Etiqueta de certificación energética\n\n Este anuncio no dispone de etiqueta de certificación energética ya que el anunciante comunica que está ""en trámite"".",Ver detalle,https://www.pisos.com/comprar/chalet-mirasierra_arroyo_del_fresno28034-3860362029_109700/#,Última actualización\n Anuncio actualizado el 22/10/2024,Ref.: -0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [482]:
df_detalles2.head(5)

,Campo1,Campo2,Campo3,Campo4,Campo5,Campo6,Título,Campo7,Campo8,Campo9,Campo10,Tiempo,Campo11,Tiempo1,Campo12,Campo13,Campo14,Campo15
0,1.900.000 €,5 habs.,6 baños,780 m²,2.435 €/m²,NaN,"Chalet en venta en Calle de Arroyofresno, 10",Ciudad Universitaria (Distrito Moncloa-Aravaca. Madrid Capital),"¿La casa de tus sueños es está quieres visitarla?La mejor zona de Madrid a 5 minutos de Moncloa acceso a M30, M40, A6, Aeropuerto a 12 minutos. La ubicación dentro de Madrid Puerta de Hierro, sentirás paz tranquilidad, pajaritos, zonas verdes, aire limpio contamos con un Sánchez Romero a 2 minutos, Mercadona a 4 minutos, Farmacia a 2 minutos, Bancos a 2 minutos, Colegios a 3 minutos, Transporte público a 1 minuto. Mi casa ahora, ¿mañana puede ser tuya? Construcción 2018, 413 m2, domótica en toda la casa placas solares, suelo radiante,y más. Con una piscina preciosa, porche, jardín, 4 habitaciones, 6 baños, una suite principal con un gran vestidor baño terraza y todas las habitaciones con baño privado y terrazas. Muy luminoso. Garaje privado para 3 - 4 coches, ascensor privado.¿Cuándo quieras venir a visitarla? disponibilidad tardes, directamente propietarios. Mas fotos privado. 1.945.000",Superficie construida: \n\t\t\t\t\t\t\t\t\t\t780 m²,Superficie construida: \n\t\t\t\t\t\t\t\t\t\t780 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie útil: \n\t\t\t\t\t\t\t\t\t\t452 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie solar: \n\t\t\t\t\t\t\t\t\t\t780 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t5\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t6\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAntigüedad: \n\t\t\t\t\t\t\t\t\t\tMenos de 5 años\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tConservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tGastos de comunidad\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\t21485502,Superficie solar: \n\t\t\t\t\t\t\t\t\t\t780 m²,Habitaciones: \n\t\t\t\t\t\t\t\t\t\t5,Superficie útil: \n\t\t\t\t\t\t\t\t\t\t452 m²,Clasificación: Disponible,Certificado energético,Anuncio actualizado el 23/10/2024,Ref.: -1
1,2.950.000 €,Obra nueva,7 habs.,8 baños,561 m²,5.258 €/m²,"Chalet en venta en Calle del Pico Milano, s/n",Mirasierra-Arroyo del Fresno (Distrito Fuencarral-El Pardo. Madrid Capital),"Parcela 2, Tipo B. Chalet de obra nueva de 561 m2 de superficie construida y situado en la zona de Madrid Capital. Está estructurado en 7 habitaciones, 8 baños completos y terraza. El Chalet tiene piscina. Aporta como extras garaje y trastero.",Superficie construida: \n\t\t\t\t\t\t\t\t\t\t561 m²,Superficie construida: \n\t\t\t\t\t\t\t\t\t\t561 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t7\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t8\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tConservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\t-0,Baños: \n\t\t\t\t\t\t\t\t\t\t8,Conservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar,Habitaciones: \n\t\t\t\t\t\t\t\t\t\t7,Clasificación: En trámite,Certificado energético,Anuncio actualizado el 24/10/2024,Ref.: -0
2,2.985.000 €,Obra nueva,7 habs.,8 baños,565 m²,5.283 €/m²,"Chalet en venta en Calle del Pico Milano, s/n",Mirasierra-Arroyo del Fresno (Distrito Fuencarral-El Pardo. Madrid Capital),"Parcela 4, Tipo A. Chalet de obra nueva de 565 m2 de superficie construida y emplazado en la zona de Madrid Capital. Está dis

In [483]:
df_detalles2.query("Campo15=='Ref.: -0'")

,Campo1,Campo2,Campo3,Campo4,Campo5,Campo6,Título,Campo7,Campo8,Campo9,Campo10,Tiempo,Campo11,Tiempo1,Campo12,Campo13,Campo14,Campo15
1,2.950.000 €,Obra nueva,7 habs.,8 baños,561 m²,5.258 €/m²,"Chalet en venta en Calle del Pico Milano, s/n",Mirasierra-Arroyo del Fresno (Distrito Fuencarral-El Pardo. Madrid Capital),"Parcela 2, Tipo B. Chalet de obra nueva de 561 m2 de superficie construida y situado en la zona de Madrid Capital. Está estructurado en 7 habitaciones, 8 baños completos y terraza. El Chalet tiene piscina. Aporta como extras garaje y trastero.",Superficie construida: \n\t\t\t\t\t\t\t\t\t\t561 m²,Superficie construida: \n\t\t\t\t\t\t\t\t\t\t561 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t7\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t8\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tConservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\t-0,Baños: \n\t\t\t\t\t\t\t\t\t\t8,Conservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar,Habitaciones: \n\t\t\t\t\t\t\t\t\t\t7,Clasificación: En trámite,Certificado energético,Anuncio actualizado el 24/10/2024,Ref.: -0


In [484]:
#Silvia, esta asignacion no es correcta, lsa columnas no tienen n unico valor por ejemplo el campo 2 tiene numero de habitaciones pero puede tener tambien la marca de si es obra nueva, etc
#df_detalles2 = df_detalles2.rename(columns={'Campo1': 'precio', 'Campo2': 'num_hab', 'Campo3': 'num_banos', 'Campo4': 'metros_construidos', 'Campo5': 'planta', 'Campo6': 'precio_por_area', 'Título': 'title', 'Campo7': 'ubicacion', 'Campo8': 'detalle', 'Campo10': 'estado', 'Campo12': 'consumo', 'Campo14': 'actualizacion', 'Campo15': 'referencia'})
#df_detalles2.tail(1)

En el dataframe de pisos tenemos las columnas, tenemos que ver que nos puede aportar el de detalle, tengo nulos en habitaciones,banos,planta,estado,barrio, ver si estos nulos los puedo sacar de detalle y  ademas añadir detalle,gastos comunidad, certificcion energetica


 0   title           4736 non-null   object        
 1   ubicacion       4736 non-null   object        
 2   precio          4736 non-null   object        
 3   habitaciones    4546 non-null   object        
 4   banos           4471 non-null   object        
 5   m2_constr       4723 non-null   object        
 6   planta          2739 non-null   object        
 7   EUR/m2          4719 non-null   object        
 8   estado          2934 non-null   object        
 9   url             4736 non-null   object        
 10  fecha_descarga  4736 non-null   datetime64[ns]
 11  tipologia       4736 non-null   object        
 12  web             4736 non-null   object        
 13  barrio          4717 non-null   object        
 14  cod_barrio      4717 non-null   object        
 15  distrito        4736 non-null   object        
 16  cod_distrito    4736 non-null   object        
 17  codigo_Postal   4717 non-null   object        
dtypes: datetime64[ns](1), object(17)

In [485]:
df_detalles1.head(2)

precio  \
0  1.945.000 €   
1  2.950.000 €   

                                                                     espacios  \
0                      5 habs.\n\t\t\t6 baños\n\t\t\t780 m²\n\t\t\t2.493 €/m²   
1  Obra nueva\n\n\t\t\t7 habs.\n\t\t\t8 baños\n\t\t\t561 m²\n\t\t\t5.258 €/m²   

                                           title  \
0   Chalet en venta en Calle de Arroyofresno, 10   
1  Chalet en venta en Calle del Pico Milano, s/n   

                                                                      distrito  \
0              Ciudad Universitaria (Distrito Moncloa-Aravaca. Madrid Capital)   
1  Mirasierra-Arroyo del Fresno (Distrito Fuencarral-El Pardo. Madrid Capital)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 detalle  \
0  ¿La casa de tus sueños es está quieres visitarla?La mejor zona de Madrid a 5 minutos de Moncloa acceso a M30, M40, A6, Aeropuerto a 12 minutos. La ubicación dentro de Madrid Puerta de Hierro, sentirás paz tranquilidad, pajaritos, zonas verdes, aire limpio contamos con un Sánchez Romero a 2 minutos, Mercadona a 4 minutos, Farmacia a 2 minutos, Bancos a 2 minutos, Colegios a 3 minutos, Transporte público a 1 minuto. Mi casa ahora, ¿mañana puede ser tuya? Construcción 2018, 413 m2, domótica en toda la casa placas solares, suelo radiante,y más. Con una piscina preciosa, porche, jardín, 4 habitaciones, 6 baños, una suite principal con un gran vestidor baño terraza y todas las habitaciones con baño privado y terrazas. Muy luminoso. Garaje privado para 3 - 4 coches, ascensor privado.¿Cuándo quieras venir a visitarla?  disponibilidad tardes, directamente propietarios. Mas fotos privado. 1.945.000   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Parcela 2, Tipo B. Chalet de obra nueva de 561  m2 de superficie construida y situado en la zona de Madrid Capital. Está estructurado en 7 habitaciones, 8 baños completos y terraza. El Chalet tiene piscina. Aporta como extras garaje y trastero.   

        Campo5  \
0  Mostrar más   
1  Mostrar más   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [486]:
df_detalles2.head(2)

,Campo1,Campo2,Campo3,Campo4,Campo5,Campo6,Título,Campo7,Campo8,Campo9,Campo10,Tiempo,Campo11,Tiempo1,Campo12,Campo13,Campo14,Campo15
0,1.900.000 €,5 habs.,6 baños,780 m²,2.435 €/m²,NaN,"Chalet en venta en Calle de Arroyofresno, 10",Ciudad Universitaria (Distrito Moncloa-Aravaca. Madrid Capital),"¿La casa de tus sueños es está quieres visitarla?La mejor zona de Madrid a 5 minutos de Moncloa acceso a M30, M40, A6, Aeropuerto a 12 minutos. La ubicación dentro de Madrid Puerta de Hierro, sentirás paz tranquilidad, pajaritos, zonas verdes, aire limpio contamos con un Sánchez Romero a 2 minutos, Mercadona a 4 minutos, Farmacia a 2 minutos, Bancos a 2 minutos, Colegios a 3 minutos, Transporte público a 1 minuto. Mi casa ahora, ¿mañana puede ser tuya? Construcción 2018, 413 m2, domótica en toda la casa placas solares, suelo radiante,y más. Con una piscina preciosa, porche, jardín, 4 habitaciones, 6 baños, una suite principal con un gran vestidor baño terraza y todas las habitaciones con baño privado y terrazas. Muy luminoso. Garaje privado para 3 - 4 coches, ascensor privado.¿Cuándo quieras venir a visitarla? disponibilidad tardes, directamente propietarios. Mas fotos privado. 1.945.000",Superficie construida: \n\t\t\t\t\t\t\t\t\t\t780 m²,Superficie construida: \n\t\t\t\t\t\t\t\t\t\t780 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie útil: \n\t\t\t\t\t\t\t\t\t\t452 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie solar: \n\t\t\t\t\t\t\t\t\t\t780 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t5\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t6\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAntigüedad: \n\t\t\t\t\t\t\t\t\t\tMenos de 5 años\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tConservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tGastos de comunidad\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\t21485502,Superficie solar: \n\t\t\t\t\t\t\t\t\t\t780 m²,Habitaciones: \n\t\t\t\t\t\t\t\t\t\t5,Superficie útil: \n\t\t\t\t\t\t\t\t\t\t452 m²,Clasificación: Disponible,Certificado energético,Anuncio actualizado el 23/10/2024,Ref.: -1
1,2.950.000 €,Obra nueva,7 habs.,8 baños,561 m²,5.258 €/m²,"Chalet en venta en Calle del Pico Milano, s/n",Mirasierra-Arroyo del Fresno (Distrito Fuencarral-El Pardo. Madrid Capital),"Parcela 2, Tipo B. Chalet de obra nueva de 561 m2 de superficie construida y situado en la zona de Madrid Capital. Está estructurado en 7 habitaciones, 8 baños completos y terraza. El Chalet tiene piscina. Aporta como extras garaje y trastero.",Superficie construida: \n\t\t\t\t\t\t\t\t\t\t561 m²,Superficie construida: \n\t\t\t\t\t\t\t\t\t\t561 m²\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t7\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t8\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tConservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar\n\t\t\t\t\t\t\t\t\n \n \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\t-0,Baños: \n\t\t\t\t\t\t\t\t\t\t8,Conservación: \n\t\t\t\t\t\t\t\t\t\tA estrenar,Habitaciones: \n\t\t\t\t\t\t\t\t\t\t7,Clasificación: En trámite,Certificado energético,Anuncio actualizado el 24/10/2024,Ref.: -0


In [487]:
df_detalles1.dropna(subset=['referencia'], inplace=True)
df_detalles1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2866 entries, 0 to 2865
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   precio           2866 non-null   object 
 1   espacios         2860 non-null   object 
 2   title            2619 non-null   object 
 3   distrito         2619 non-null   object 
 4   detalle          2866 non-null   object 
 5   Campo5           2789 non-null   object 
 6   caracteristicas  2866 non-null   object 
 7   Campo7           2715 non-null   object 
 8   consumo          2866 non-null   object 
 9   Campo9_texto     2866 non-null   object 
 10  url              2866 non-null   object 
 11  actualizacion    2866 non-null   object 
 12  referencia       2866 non-null   object 
 13  Campo12          0 non-null      float64
 14  Campo13          0 non-null      float64
 15  Campo14          0 non-null      float64
 16  Campo15          0 non-null      float64
 17  Campo16       

In [488]:
#df_detalles2.dropna(subset=['referencia'], inplace=True)
#df_detalles2.info()

In [489]:
df_detalles1.shape, df_detalles2.shape

((2866, 21), (2805, 18))

In [490]:
# antes de hacer el merge hay que quedarse con los valores unicos de referencia, creo 2 nuevos ficheros dandole valores de referencia diferente a viviendas de la misma promocion
#finalmente no hacemos merge, viendo los dataset nos quedamos solo con el 1, el 2 o nos aporta nada.

In [491]:
#df_detalles = pd.merge(df_detalles1, df_detalles2, on='referencia', how='outer')
#df_detalles.info()

In [492]:
#df_detalles = df_detalles[['precio_x', 'precio_y', 'espacios', 'title_x', 'title_y', 'distrito', 'detalle_x', 'detalle_y', 
                           #'caracteristicas', 'consumo_x', 'consumo_y', 'url', 'actualizacion_x', 'actualizacion_y', 
                           #'referencia', 'num_hab', 'num_banos', 'metros_construidos', 'planta', 'precio_por_area', 'ubicacion', 'estado']]
#df_detalles.columns

In [493]:
#df_detalles.sort_values(by='referencia', ascending=True).head(10)

In [494]:
#df_detalles['ubicacion'] = df_detalles['ubicacion'].fillna('/')
#df_detalles['distrito'] = df_detalles['distrito'].fillna('/')

In [495]:
#df_detalles['ubicacion'] = df_detalles['ubicacion'] + ' / ' + df_detalles['distrito']
#df_detalles.columns

In [496]:
#df_detalles.head(10)

In [497]:
#df_detalles = df_detalles.drop(columns=['precio_y', 'title_y', 'detalle_y', 'consumo_x', 'actualizacion_x', 'distrito'], axis=1)
#df_detalles = df_detalles.rename(columns={'precio_x': 'precio', 'title_x': 'title', 'detalle_x': 'detalle', 'consumo_y': 'consumo', 'actualizacion_y': 'actualizacion'})  
df_detalles=df_detalles1[["title","precio","espacios","distrito","detalle","caracteristicas","consumo","url","actualizacion","referencia"] ]
#df_detalles.tail(1)

In [498]:
df_detalles['fecha_descarga'] = pd.to_datetime('2024-10-19')
df_detalles['web'] = 'pisos.com'
df_detalles.columns

C:\Users\extas\AppData\Local\Temp\ipykernel_9684\1931937819.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detalles['fecha_descarga'] = pd.to_datetime('2024-10-19')
C:\Users\extas\AppData\Local\Temp\ipykernel_9684\1931937819.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detalles['web'] = 'pisos.com'


Index(['title', 'precio', 'espacios', 'distrito', 'detalle', 'caracteristicas',
       'consumo', 'url', 'actualizacion', 'referencia', 'fecha_descarga',
       'web'],
      dtype='object')

In [499]:
df_detalles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2866 entries, 0 to 2865
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   title            2619 non-null   object        
 1   precio           2866 non-null   object        
 2   espacios         2860 non-null   object        
 3   distrito         2619 non-null   object        
 4   detalle          2866 non-null   object        
 5   caracteristicas  2866 non-null   object        
 6   consumo          2866 non-null   object        
 7   url              2866 non-null   object        
 8   actualizacion    2866 non-null   object        
 9   referencia       2866 non-null   object        
 10  fecha_descarga   2866 non-null   datetime64[ns]
 11  web              2866 non-null   object        
dtypes: datetime64[ns](1), object(11)
memory usage: 268.8+ KB


In [500]:
df_detalles = df_detalles.reset_index(drop=True)

### TRANSFORMAR DATOS

In [501]:
precio_decimales = df_detalles[df_detalles['precio'].str.contains(',', na=False)]
precio_decimales
# Se ha comprobado que ningún 'precio' contiene decimales.

,title,precio,espacios,distrito,detalle,caracteristicas,consumo,url,actualizacion,referencia,fecha_descarga,web


In [502]:
#area_decimales = df_detalles[df_detalles['precio_por_area'].str.contains(',', na=False)]
#area_decimales
# Se ha comprobado que ningún 'precio_por_area' contiene decimales.

In [503]:
#metros_decimales = df_detalles[df_detalles['metros_construidos'].str.contains(',', na=False)]
#metros_decimales
# Se ha comprobado que ningún 'metros_construidos' contiene decimales.

In [504]:

df_detalles['precio'] = df_detalles['precio'].str.replace(r'\D', '', regex=True)
df_detalles['precio'] = df_detalles['precio'].astype(float)


In [505]:
df_detalles.columns

Index(['title', 'precio', 'espacios', 'distrito', 'detalle', 'caracteristicas',
       'consumo', 'url', 'actualizacion', 'referencia', 'fecha_descarga',
       'web'],
      dtype='object')

In [506]:

#renombre estado a caracteristicas_1
#######################3
#####################33
#######################
###revisaresto


#dejo la columna espacios para utilizarla en cuando haga el merge con el pisos general si tuvier algun dato que no tenemos #
#df_detalles['metros_construidos'] = df_detalles['metros_construidos'].str.replace(r'\D', '', regex=True)
#df_detalles['precio_por_area'] = df_detalles['precio_por_area'].str.replace(r'\D', '', regex=True)
#df_detalles['num_hab'] = df_detalles['num_hab'].str.replace(r'\D', '', regex=True)
#df_detalles['num_banos'] = df_detalles['num_banos'].str.replace(r'\D', '', regex=True)
#df_detalles['planta'] = df_detalles['planta'].str.replace(r'\D', '', regex=True)
df_detalles['actualizacion'] = df_detalles['actualizacion'].astype(str)
df_detalles['actualizacion'] = df_detalles['actualizacion'].str.replace('Última actualización','',regex=False)
df_detalles['actualizacion'] = df_detalles['actualizacion'].str.replace('Anuncio actualizado el','',regex=False)
# Convertir la columna 'actualizacion' a tipo datetime
df_detalles['actualizacion'] = df_detalles['actualizacion'].astype('datetime64[ns]')
# Eliminar el carácter '#' de la columna 'url'
df_detalles['url'] = df_detalles['url'].str.replace('#', '', regex=False)
df_detalles.head(1)

title     precio  \
0  Chalet en venta en Calle de Arroyofresno, 10  1945000.0   

                                                 espacios  \
0  5 habs.\n\t\t\t6 baños\n\t\t\t780 m²\n\t\t\t2.493 €/m²   

                                                          distrito  \
0  Ciudad Universitaria (Distrito Moncloa-Aravaca. Madrid Capital)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 detalle  \
0  ¿La casa de tus sueños es está quieres visitarla?La mejor zona de Madrid a 5 minutos de Moncloa acceso a M30, M40, A6, Aeropuerto a 12 minutos. La ubicación dentro de Madrid Puerta de Hierro, sentirás paz tranquilidad, pajaritos, zonas verdes, aire limpio contamos con un Sánchez Romero a 2 minutos, Mercadona a 4 minutos, Farmacia a 2 minutos, Bancos a 2 minutos, Colegios a 3 minutos, Transporte público a 1 minuto. Mi casa ahora, ¿mañana puede ser tuya? Construcción 2018, 413 m2, domótica en toda la casa placas solares, suelo radiante,y más. Con una piscina preciosa, porche, jardín, 4 habitaciones, 6 baños, una suite principal con un gran vestidor baño terraza y todas las habitaciones con baño privado y terrazas. Muy luminoso. Garaje privado para 3 - 4 coches, ascensor privado.¿Cuándo quieras venir a visitarla?  disponibilidad tardes, directamente propietarios. Mas fotos privado. 1.945.000   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [507]:
df_detalles[["caracteristicas"]].sample(10)

caracteristicas
786                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Características\n\t\t\n                    Datos básicos\n                    \n                            \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie construida: \n\t\t\t\t\t\t\t\t\t\t100 m²\n\t\t\t\t\t\t\t\t\n                            \n                            \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tSuperficie útil: \n\t\t\t\t\t\t\t\t\t\t100 m²\n\t\t\t\t\t\t\t\t\n                            \n                            \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tHabitaciones: \n\t\t\t\t\t\t\t\t\t\t2\n\t\t\t\t\t\t\t\t\n                            \n                            \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tBaños: \n\t\t\t\t\t\t\t\t\t\t2\n\t\t\t\t\t\t\t\t\n                            \n                            \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAntigüedad: \n\t\t\t\t\t\t\t\t\t\tMás de 50 años\n\t\t\t\t\t\t\t\t\n                            \n                            \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tReferencia: \n\t\t\t\t\t\t\t\t\t\tEP693-admiP-513\n\t\t\t\t\t\t\t\t\n                            \n                    \n                    Muebles y acabados\n                    \n                            \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAmueblado\n\t\t\t\t\t\t\t\t\n                            \n                    \n                    Equipamiento e instalaciones\n                    \n                            \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tAire acondicionado\n\t\t\t\t\t\t\t\t\n                            \n                            \n\t\t\t\t\

In [508]:
#Lo primero miro la columna consumo, para ver si está disponible o en tramite, entre los que esten disponibles sacamos la letra
# buscar en la columna consumo "Disponible" ó "En tramite"

In [509]:

def comprobar_texto_en_columna(df, columna, texto):
    """
    Comprueba si un texto está presente en los valores de una columna de un DataFrame.

    Parámetros:
    df (pd.DataFrame): El DataFrame en el que buscar.
    columna (str): El nombre de la columna en la que buscar.
    texto (str): El texto a buscar.

    Devuelve:
    int: 1 si el texto está presente en cualquier valor de la columna, 0 si no está presente.
    """
    if columna in df.columns:
        patron = r'\b' + re.escape(texto) + r'\b'
        return df[columna].apply(lambda x: 1 if re.search(patron, x) else 0)
        #return 1 if df[columna].str.contains(texto).any() else 0
        #return 1 if df[columna].str.contains(r'\b' + re.escape(texto) + r'\b').any() else 0
 
    else:
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame.")



In [510]:
df_detalles['disponible_ce'] = comprobar_texto_en_columna(df_detalles, 'consumo', 'Clasificación: Disponible')

In [511]:
df_detalles.query("disponible_ce==1")

title  \
0                             Chalet en venta en Calle de Arroyofresno, 10   
22                                              Piso en venta en Trafalgar   
112                                               Piso en venta en Adelfas   
113                                              Piso en venta en Numancia   
114                                             Ático en venta en Vallecas   
...                                                                    ...   
2760                                       Piso en venta en Cortes-Huertas   
2780  Piso en venta en Calle de Fernán González, cerca de Calle Jorge Juan   
2781  Piso en venta en Calle de Fernán González, cerca de Calle Jorge Juan   
2817                                Piso en venta en Calle de Hartzenbusch   
2829                      Piso en venta en Paseo de San Francisco de Sales   

         precio  \
0     1945000.0   
22     850000.0   
112    815000.0   
113    380000.0   
114    255000.0   
...         ...   
2760   539000.0   
2780   700000.0   
2781   590000.0   
2817   820000.0   
2829  1069000.0   

                                                                      espacios  \
0                       5 habs.\n\t\t\t6 baños\n\t\t\t780 m²\n\t\t\t2.493 €/m²   
22     2 habs.\n\t\t\t2 baños\n\t\t\t127 m²\n\t\t\t1ª planta\n\t\t\t6.692 €/m²   
112   3 habs.\n\t\t\t2 baños\n\t\t\t132 m²\n\t\t\t12ª planta\n\t\t\t6.174 €/m²   
113     3 habs.\n\t\t\t2 baños\n\t\t\t98 m²\n\t\t\t2ª planta\n\t\t\t3.877 €/m²   
114      2 habs.\n\t\t\t1 baño\n\t\t\t76 m²\n\t\t\t3ª planta\n\t\t\t3.355 €/m²   
...                                                                        ...   
2760      1 hab.\n\t\t\t1 baño\n\t\t\t63 m²\n\t\t\t1ª planta\n\t\t\t8.555 €/m²   
2780                     3 habs.\n\t\t\t2 baños\n\t\t\t97 m²\n\t\t\t7.216 €/m²   
2781         2 habs.\n\t\t\t1 baño\n\t\t\t82 m²\n\t\t\tBajo \n\t\t\t7.195 €/m²   
2817                     2 habs.\n\t\t\t2 baños\n\t\t\t95 m²\n\t\t\t8.631 €/m²   
2829                    4 habs.\n\t\t\t4 baños\n\t\t\t179 m²\n\t\t\t5.972 €/m²   

                                                                      distrito  \
0              Ciudad Universitaria (Distrito Moncloa-Aravaca. Madrid Capital)   
22                               Trafalgar (Distrito Chamberí. Madrid Capital)   
112                                  Adelfas (Distrito Retiro. Madrid Capital)   
113                     Numancia (Distrito Puente de Vallecas. Madrid Capital)   
114   Casco Histórico de Vallecas (Distrito Villa de Vallecas. Madrid Capital)   
...                                                                        ...   
2760                          Cortes-Huertas (Distrito Centro. Madrid Capital)   
2780                                 Goya (Distrito Salamanca. Madrid Capital)   
2781                                 Goya (Distrito Salamanca. Madrid Capital)   
2817                             Trafalgar (Distrito Chamberí. Madrid Capital)   
2829                          Vallehermoso (Distrito Chamberí. Madrid Capital)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [512]:
 #
#patron = r'Consumo:\s*(\d+\s*kWh/m²\s*año)'
 #df['consumo_extraido'] = df[columna].apply(lambda x: re.search(patron, x).group(1) if re.search(patron, x) else None)
   

In [513]:
def extraer(texto):
    # Inicializar valores
    consumo_ce = None
    letra_ce = None

    # Buscar valor de consumo
    consumo_match = re.search(r'Consumo:\s*(\d+\s*kWh/m²\s*año)', texto)
    if consumo_match:
        consumo_ce = consumo_match.group(1)

        # Buscar valor de letra_ce solo si disponible_ce es 1
    letra_ce_match = re.search(r'Consumo:\s*\d+\s*kWh/m²\s*año\s*\n\s*(\w)', texto)
    if letra_ce_match:
        letra_ce = letra_ce_match.group(1)

    return pd.Series([consumo_ce, letra_ce])

   

In [514]:
df_detalles.shape

(2866, 13)

In [515]:
df_detalles[['consumo_ce', 'letra_ce']] = df_detalles['consumo'].apply(extraer)
  

In [516]:
df_detalles["disponible_ce"].value_counts(dropna=False)

disponible_ce
0    2255
1     611
Name: count, dtype: int64

In [517]:
df_detalles.query("letra_ce.notnull()").count()

title              544
precio             611
espacios           610
distrito           544
detalle            611
caracteristicas    611
consumo            611
url                611
actualizacion      611
referencia         611
fecha_descarga     611
web                611
disponible_ce      611
consumo_ce         611
letra_ce           611
dtype: int64

In [518]:
df_detalles.query("consumo_ce.notnull()").count()

title              544
precio             611
espacios           610
distrito           544
detalle            611
caracteristicas    611
consumo            611
url                611
actualizacion      611
referencia         611
fecha_descarga     611
web                611
disponible_ce      611
consumo_ce         611
letra_ce           611
dtype: int64

In [519]:
df_detalles["consumo_ce"].value_counts(dropna=False)

consumo_ce
None              2255
1 kWh/m² año        43
197 kWh/m² año      21
104 kWh/m² año      20
300 kWh/m² año      12
                  ... 
63 kWh/m² año        1
356 kWh/m² año       1
348 kWh/m² año       1
253 kWh/m² año       1
72 kWh/m² año        1
Name: count, Length: 264, dtype: int64

In [520]:
df_detalles["letra_ce"].value_counts(dropna=False)

letra_ce
None    2255
e        332
d        126
f         46
c         46
g         28
a         18
b         15
Name: count, dtype: int64

In [521]:
#me quedo con letra_ce y elimino consumo_ce y consumo
df_detalles=df_detalles.drop(columns=["consumo_ce","consumo"])

## Ahora extraemos caracteriscas

In [522]:
def extraer_info_generica(texto, clave):
    if clave in texto:
        return texto.split(clave)[1].split("\n")[0].strip()
    return None

In [523]:
def extraer_info_generica_texto(texto, clave):
    if clave in texto:
        return texto.split(clave)[1].split("\n")[0].strip()
    return None

In [524]:
def extraer_info_generica(df, columna_origen, claves):
    for nueva_columna, clave in claves.items():
        df[nueva_columna] = df[columna_origen].apply(lambda texto: extraer_info_generica_texto(texto, clave))


In [525]:

# Diccionario con las claves a extraer de la columna 'estado'


# Diccionario con las claves a extraer de la columna 'caracteristicas'
claves_caracteristicas = {
    'orientacion': 'Orientación:',
    'jardin': 'Jardín:',
    'aire_acondicionado': 'Aire acondicionado:',
    'calefaccion': 'Calefacción:',
    'cocina': 'Cocina equipada:',
    'tipo_suelo': 'Tipo suelo:',
    'piscina': 'Piscina:',
    'armarios': 'Armarios empotrados:',
    'garaje': 'Garaje:',
    'm2_constr':'Superficie construida',
    'habitaciones':'Habitaciones',
    'banos':'Baños',
    'planta':'Planta',
    'conservacion': 'Conservación:',
    'm2_utiles': 'Superficie útil:',
    'antiguedad': 'Antigüedad:',
    'gastos_comunidad': 'Gastos de comunidad:',
    'interior_exterior': 'Planta:',
    'estado': 'Conservación:',
    'm2_utiles': 'Superficie útil:',
    'antiguedad': 'Antigüedad:',   
    'puerta_blindada': 'Puerta blindada:',
    'alarma': 'Sistema de seguridad:',
    'antiguedad': 'Antigüedad:',   
    'acceso_discapacitado':'Adaptado a personas con movilidad reducida'
    
    
}

# Aplicar la función genérica a las columnas correspondientes
#extraer_info_generica(df_detalles, 'estado', claves_estado)
extraer_info_generica(df_detalles, 'caracteristicas', claves_caracteristicas)


In [526]:
# Lista de características a buscar
lista_caracteristicas = ['Terraza', 'Trastero', 'Ascensor', 'Balcón','Exterior','Amueblado','Seguridad 24 h','Vidrios dobles']
# Crear una columna para cada característica
for caracteristica in lista_caracteristicas:
    df_detalles[caracteristica] = df_detalles['caracteristicas'].str.contains(fr'\b{caracteristica}\b', regex=True, na=False).replace({True: 1, False: None})

C:\Users\extas\AppData\Local\Temp\ipykernel_9684\2678210444.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_detalles[caracteristica] = df_detalles['caracteristicas'].str.contains(fr'\b{caracteristica}\b', regex=True, na=False).replace({True: 1, False: None})
C:\Users\extas\AppData\Local\Temp\ipykernel_9684\2678210444.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_detalles[caracteristica] = df_detalles['caracteristicas'].str.contains(fr'\b{caracteristica}\b', regex=True, na=False).replace({True: 1, False: None})
C:\U

In [527]:
def normalizar_nombres_columnas(df):
    df.columns = [unidecode.unidecode(col).lower() for col in df.columns]

In [528]:
normalizar_nombres_columnas(df_detalles)
df_detalles.columns

Index(['title', 'precio', 'espacios', 'distrito', 'detalle', 'caracteristicas',
       'url', 'actualizacion', 'referencia', 'fecha_descarga', 'web',
       'disponible_ce', 'letra_ce', 'orientacion', 'jardin',
       'aire_acondicionado', 'calefaccion', 'cocina', 'tipo_suelo', 'piscina',
       'armarios', 'garaje', 'm2_constr', 'habitaciones', 'banos', 'planta',
       'conservacion', 'm2_utiles', 'antiguedad', 'gastos_comunidad',
       'interior_exterior', 'estado', 'puerta_blindada', 'alarma',
       'acceso_discapacitado', 'terraza', 'trastero', 'ascensor', 'balcon',
       'exterior', 'amueblado', 'seguridad 24 h', 'vidrios dobles'],
      dtype='object')

In [529]:
df_detalles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2866 entries, 0 to 2865
Data columns (total 43 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   title                 2619 non-null   object        
 1   precio                2866 non-null   float64       
 2   espacios              2860 non-null   object        
 3   distrito              2619 non-null   object        
 4   detalle               2866 non-null   object        
 5   caracteristicas       2866 non-null   object        
 6   url                   2866 non-null   object        
 7   actualizacion         2866 non-null   datetime64[ns]
 8   referencia            2866 non-null   object        
 9   fecha_descarga        2866 non-null   datetime64[ns]
 10  web                   2866 non-null   object        
 11  disponible_ce         2866 non-null   int64         
 12  letra_ce              611 non-null    object        
 13  orientacion       

# Extraer la fecha de actualización
#formato_fecha = r'(\d{2}/\d{2}/\d{4})'
#df_detalles['fecha'] = df_detalles['actualizacion'].str.extract(formato_fecha)

# Eliminar el texto 'Clasificación: ' de la columna 'consumo'
#df_detalles['consumo'] = df_detalles['consumo'].str.replace('Clasificación: ', '', regex=False)
# Extraer texto de la columna estado
df_detalles['conservacion'] = df_detalles['estado'].str.extract(r'Conservación: \s*(.*?)\s*(?:\n|$)')
df_detalles['metros_utiles'] = df_detalles['estado'].str.extract(r'Superficie útil: \s*(.*?)\s*(?:\n|$)')
df_detalles['antiguedad'] = df_detalles['estado'].str.extract(r'Antigüedad: \s*(.*?)\s*(?:\n|$)')
df_detalles['comunidad'] = df_detalles['estado'].str.extract(r'Gastos de comunidad: \s*(.*?)\s*(?:\n|$)')
df_detalles['interior_exterior'] = df_detalles['estado'].str.extract(r'Planta:\s*.*?\n\s*(.*?)\n\s*Antigüedad')
df_detalles['interior_exterior'] = df_detalles['interior_exterior'].str.strip() # Limpiar espacios en blanco

# Extraer texto de la columna características
df_detalles['orientacion'] = df_detalles['caracteristicas'].str.extract(r'Orientación: \s*(.*?)\s*(?:\n|$)')
df_detalles['jardin'] = df_detalles['caracteristicas'].str.extract(r'Jardín: \s*(.*?)\s*(?:\n|$)')
df_detalles['aire_acond'] = df_detalles['caracteristicas'].str.extract(r'Aire acondicionado: \s*(.*?)\s*(?:\n|$)')
df_detalles['calefaccion'] = df_detalles['caracteristicas'].str.extract(r'Calefacción: \s*(.*?)\s*(?:\n|$)')
df_detalles['cocina'] = df_detalles['caracteristicas'].str.extract(r'Cocina equipada: \s*(.*?)\s*(?:\n|$)')
df_detalles['suelo'] = df_detalles['caracteristicas'].str.extract(r'Tipo suelo: \s*(.*?)\s*(?:\n|$)')
df_detalles['piscina'] = df_detalles['caracteristicas'].str.extract(r'Piscina: \s*(.*?)\s*(?:\n|$)')
df_detalles['armarios'] = df_detalles['caracteristicas'].str.extract(r'Armarios empotrados: \s*(.*?)\s*(?:\n|$)')
df_detalles['garaje'] = df_detalles['caracteristicas'].str.extract(r'Garaje: \s*(.*?)\s*(?:\n|$)')





In [530]:
df_detalles = df_detalles.drop([ 'caracteristicas'], axis=1)
df_detalles.tail(1)

,title,precio,espacios,distrito,detalle,url,actualizacion,referencia,fecha_descarga,web,disponible_ce,letra_ce,orientacion,jardin,aire_acondicionado,calefaccion,cocina,tipo_suelo,piscina,armarios,garaje,m2_constr,habitaciones,banos,planta,conservacion,m2_utiles,antiguedad,gastos_comunidad,interior_exterior,estado,puerta_blindada,alarma,acceso_discapacitado,terraza,trastero,ascensor,balcon,exterior,amueblado,seguridad 24 h,vidrios dobles
2865,Piso en venta en Prosperidad,3300000.0,3 habs.\n\t\t\t6 baños\n\t\t\t301 m²\n\t\t\t2ª planta\n\t\t\t10.963 €/m²,El Viso (Distrito Chamartín. Madrid Capital),"¡Exclusivo Piso de Lujo en El Viso, Madrid! Presentamos una oportunidad única para los amantes del lujo: un exclusivo piso en uno de los barrios más prestigiosos de Madrid, El Viso. Este moderno edificio, diseñado por el reconocido arquitecto Joaquín Torres, cuenta con solo 8 viviendas, garantizando privacidad y exclusividad. El piso ocupa toda la planta, por lo que no tiene vecinos laterales. El ascensor privado ofrece acceso directo a un recibidor con aseo de cortesía, permitiendo entrar de manera independiente a la cocina, al salón o al área de dormitorios. Distribución: La elegante y luminosa cocina en isla tiene acceso al comedor y a la terraza, se encuentra completamente equipada con electrodomésticos de alta gama, tiene un con baño, lavadero y zona de servicio. El imponente Salón-Comedor es un espacio elegante pero a la vez cálido y acogedor, con mucha luz y con acceso a una magnífica terraza con impresionantes vistas. A la zona de dormitorios se accede desde el salón o desde el recibidor a través de un pasillo que le brinda intimidad con su acceso independiente. El primero de estos tiene dos armarios grandes y un baño en suite con plato de ducha. El segundo dormitorio tiene un vestidor y baño en suite con bañera de hidromasajes. El majestuoso dormitorio principal tiene un baño con plato de ducha, un vestidor con baño con bañera de hidromasajes y otro vestidor. Este singular piso es totalmente exterior, con abundante luz natural incluso en los días más nublados, sus calidades constructivas son materiales de primera, aire acondicionado central y calefacción por aerotermia. Ventanas Climalit de doble cristal con excelente aislamiento térmico y acústico en todas sus estancias. Tiene dos amplias plazas de garaje. El Viso es una zona consolidada con viviendas de alto standing, embajadas y consulados, lo que garantiza la mayor seguridad por contar con vigilancia policial permanente. Dispone de reconocidos restaurantes, el Hospital Universitario San Francisco de Asís, el EAE Business School, supermercados, y está muy bien comunicada con calles emblemáticas como Velázquez, Serrano, López de Hoyos, Paseo de la Castellana y Príncipe de Vergara. ¡No pierda la oportunidad de vivir en este enclave de lujo y exclusividad! Contáctenos para más información y para concertar una visita.",https://www.pisos.com/comprar/piso-chamartin_el_viso28002-45915053595_108900/,2024-10-17,Ref.: XA856-1025,2024-10-19,pisos.com,0,None,,None,,,None,None,None,None,,:,:,:,:,None,None,None,,,None,None,None,None,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN


In [461]:
utiles_decimales = df_detalles[df_detalles['m2_utiles'].str.contains(',', na=False)]
utiles_decimales
# Se ha comprobado que ningún 'metros_utiles' contiene decimales.

,title,precio,espacios,distrito,detalle,url,actualizacion,referencia,fecha_descarga,web,disponible_ce,letra_ce,orientacion,jardin,aire_acondicionado,calefaccion,cocina,tipo_suelo,piscina,armarios,garaje,m2_constr,habitaciones,banos,planta,conservacion,m2_utiles,antiguedad,gastos_comunidad,interior_exterior,puerta_blindada,alarma,acceso_discapacitado,Terraza,Trastero,Ascensor,Balcón,Exterior,Amueblado,Seguridad 24 h,Vidrios Dobles,Vidrios Triples,Vidrios dobles,Vidrios triples,estado


In [ ]:
#df_detalles['metros_utiles'] = df_detalles['metros_utiles'].str.replace(r'\D', '', regex=True)
#df_detalles['fecha'] = df_detalles['fecha'].str.replace(r'(\d{2})/(\d{2})/(\d{4})', r'\3-\2-\1', regex=True)
#df_detalles['fecha'] = pd.to_datetime(df_detalles['fecha'])


In [532]:
df_detalles.shape
# (3033, 34)

(2866, 42)

In [533]:
df_detalles_dupl = df_detalles.duplicated().sum()
df_detalles_dupl
# 5 filas duplicadas

0

## COLUMNAS DE DF DETALLES

In [ ]:
#df_detalles.rename(columns={'num_hab': 'habitaciones', 'num_banos': 'banos', 'metros_construidos': 'm2_constr', 'precio_por_area': 'EUR/m2', 'consumo': 'consumo_energia','fecha': 'fecha_publicacion', 'metros_utiles': 'm2_utiles', 'antiguedad': 'ano_construccion', 'comunidad': '€_comunidad_mes', 'aire_acond': 'aire_acondicionado', 'suelo': 'tipo_suelo', 'Terraza': 'terraza', 'Trastero': 'trastero', 'Ascensor': 'ascensor', 'Balcón': 'balcon'}, inplace=True)
#df_detalles.columns

Index(['title', 'ubicacion', 'precio', 'habitaciones', 'banos', 'm2_constr',
       'planta', 'EUR/m2', 'url', 'fecha_descarga', 'web', 'detalle',
       'consumo_energia', 'referencia', 'fecha_publicacion', 'conservacion',
       'm2_utiles', 'ano_construccion', '€_comunidad_mes', 'interior_exterior',
       'orientacion', 'jardin', 'aire_acondicionado', 'calefaccion', 'cocina',
       'tipo_suelo', 'piscina', 'armarios', 'garaje', 'terraza', 'trastero',
       'ascensor', 'balcon'],
      dtype='object')

In [535]:
df_detalles = df_detalles.reset_index(drop=True)
df_detalles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2866 entries, 0 to 2865
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   title                 2619 non-null   object        
 1   precio                2866 non-null   float64       
 2   espacios              2860 non-null   object        
 3   distrito              2619 non-null   object        
 4   detalle               2866 non-null   object        
 5   url                   2866 non-null   object        
 6   actualizacion         2866 non-null   datetime64[ns]
 7   referencia            2866 non-null   object        
 8   fecha_descarga        2866 non-null   datetime64[ns]
 9   web                   2866 non-null   object        
 10  disponible_ce         2866 non-null   int64         
 11  letra_ce              611 non-null    object        
 12  orientacion           1159 non-null   object        
 13  jardin            

In [536]:
url_duplicadas = df_detalles['url'].duplicated().sum()
url_duplicadas

0

In [537]:
df_detalles = df_detalles.drop_duplicates(subset=['url'])
df_detalles.shape
# (2867, 33)

(2866, 42)

In [538]:
df_detalles.to_csv('pisoscom_detalles_limpio.csv',  sep=';',index=False)